In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Optimized auto_polysegment_with_pSMILES_fixed_amide.py
セグメント抽出ルールを強化したバージョン（修正版）
"""

import random, pandas as pd
from collections import Counter
from rdkit import Chem
from rdkit.Chem import rdChemReactions

# --- 0. ユーザー定義 ---
base_smiles = {
    'A': "O=C(O)c1ccc(O)cc1",
    'B': "Oc1ccc(O)cc1",
    'C': "O=C(O)c1ccc(C(=O)O)cc1",
    'D': "C1=C2C(C=C(C(O)=O)C=C2)=CC=C1C(O)=O",
    'E': "Oc1ccc(N)cc1"

}
feed_ratios = {'A':0.5,'B':0.125,'C':0.2,'D':0.05,'E':0.125}

N_CHAINS, CHAIN_LEN = 10000, 100
TARGET_CUM = 0.95
random.seed(0)

# --- 1. SMILESと官能基キャッシュ ---
def ends_from_smiles(smi):
    mol = Chem.MolFromSmiles(smi)
    acid = Chem.MolFromSmarts("[CX3](=O)[OX2H1]")
    phen = Chem.MolFromSmarts("[c]O")
    amine = Chem.MolFromSmarts("[NH2]")
    hits = []
    for patt, lab in [(acid, 'a'), (phen, 'b'), (amine, 'c')]:
        for match in mol.GetSubstructMatches(patt):
            if lab == 'a':  # カルボン酸: 炭素原子
                idx = match[0]
            elif lab == 'b':  # フェノール: 酸素原子
                idx = match[1]
            else:  # アミン: 窒素原子
                idx = match[0]
            hits.append((idx, lab))
    hits.sort(key=lambda x: x[0])
    if len(hits) < 2:
        raise ValueError(f"端基検出失敗: {smi}")
    return hits[0][1], hits[-1][1]

monomer_names = list(base_smiles.keys())
ends_cache = {}
for L, smi in base_smiles.items():
    try:
        ends_cache[L] = ends_from_smiles(smi)
    except Exception as e:
        print(f"エラー: {e}")
        ends_cache[L] = ('a', 'a')

MONOMERS, MON_DEF, sym_factor = {}, {}, {}
token2fg = {'a': 'COOH', 'b': 'PhOH', 'c': 'NH2'}

for L in monomer_names:
    l, r = ends_cache[L]
    smi = base_smiles[L]
    
    for key in [f"{l}-{L}-{r}", f"{r}-{L}-{l}"]:
        MONOMERS[key] = {
            "smiles": smi,
            "ends": [token2fg[key[0]], token2fg[key[-1]]]
        }
        MON_DEF[key] = (key[0], key[-1])
    
    sym_factor[L] = 2 if l == r else 1

REACTION_RULES = {'a': ['b', 'c'], 'b': ['a'], 'c': ['a']}

# --- 2. チェーン生成 ---
def pick_letter(weights):
    total = sum(weights.values())
    r = random.random() * total
    acc = 0
    for L, w in weights.items():
        acc += w
        if r <= acc:
            return L
    return list(weights.keys())[-1]

def grow_chain():
    chain = []
    first_weights = {L: feed_ratios[L] * sym_factor[L] for L in feed_ratios}
    firstL = pick_letter(first_weights)
    l, r = ends_cache[firstL]
    
    if l == r and random.random() < 0.5:
        key = f"{r}-{firstL}-{l}"
    else:
        key = f"{l}-{firstL}-{r}"
    
    chain.append(key)
    end_token = MON_DEF[key][1]
    
    while len(chain) < CHAIN_LEN:
        cand = {}
        for L in base_smiles:
            l0, r0 = ends_cache[L]
            w = feed_ratios[L] * sym_factor[L]
            
            if l0 in REACTION_RULES[end_token]:
                cand[f"{l0}-{L}-{r0}"] = w
            if r0 in REACTION_RULES[end_token] and l0 != r0:
                cand[f"{r0}-{L}-{l0}"] = w
        
        if not cand:
            end_token = 'a' if end_token != 'a' else 'b'
            continue
            
        key = pick_letter(cand)
        chain.append(key)
        end_token = MON_DEF[key][1]
    
    return chain

# --- 3. セグメント抽出（修正版）---
# --- 3. セグメント抽出（修正版）---
def tok_chain(chain):
    return [tok for seg in chain for tok in seg.split('-')]

def canon(tokens):
    return min('-'.join(tokens), '-'.join(tokens[::-1]))

def extract(tokens):
    segs = []
    n = len(tokens)
    
    # モノマー境界を検出
    mono_starts = []
    for i in range(0, n-2, 1):
        if tokens[i] in ['a','b','c'] and tokens[i+1] in monomer_names and tokens[i+2] in ['a','b','c']:
            mono_starts.append(i)
    
    # 各モノマー開始点からセグメントを抽出
    for start_idx in mono_starts:
        start_tok = tokens[start_idx]
        
        # 開始トークンに基づいて終了トークンを決定
        if start_tok in ['b', 'c']:
            end_target = 'a'
        else:  # 'a'
            # 修正: a開始の場合、終端はbまたはc
            end_target = ['b', 'c']
        
        # 次のモノマー境界から探索
        for next_start in mono_starts:
            if next_start <= start_idx:
                continue
                
            # 終端トークン位置 (モノマーの終端)
            end_idx = next_start + 2
            
            if end_idx >= n:
                break
                
            # 修正: 終端トークンのチェックを柔軟に
            if (isinstance(end_target, str) and tokens[end_idx] == end_target) or \
               (isinstance(end_target, list) and tokens[end_idx] in end_target):
                # 少なくとも2つの完全なモノマーを含む
                seg_tokens = tokens[start_idx:end_idx+1]
                segs.append(canon(seg_tokens))
                break
    
    return segs
# --- 4. 化学反応処理 ---
rxn_ester = rdChemReactions.ReactionFromSmarts("[C:1](=O)[O&H1:2].[c:3][O&H1:4]>>[C:1](=O)[O:4][c:3]")
rxn_amide = rdChemReactions.ReactionFromSmarts("[C:1](=O)[O&H1:2].[NH2:3]>>[C:1](=O)[NH:3]")
rx_acid = rdChemReactions.ReactionFromSmarts("[C:1](=O)[O&H1:2]>>[C:1](=O)[O:2][*:3]")
rx_ph = rdChemReactions.ReactionFromSmarts("[c:1][O&H1:2]>>[c:1][O:2][*:3]")
rx_amine = rdChemReactions.ReactionFromSmarts("[NH2:1]>>[NH:1][*:2]")

def get_largest_fragment(mol):
    frags = Chem.GetMolFrags(mol, asMols=True)
    return max(frags, key=lambda x: x.GetNumAtoms()) if frags else mol

# --- 4. 化学反応処理（結合順序を考慮した修正版）---
def build_polymer(seq):
    if not seq:
        return ""
    
    # モノマー列から分子オブジェクトと端基情報を取得
    mols = []
    left_ends = []
    right_ends = []
    
    for key in seq:
        mol = Chem.MolFromSmiles(MONOMERS[key]["smiles"])
        if mol is None:
            return ""
        mols.append(mol)
        
        # モノマーの左端と右端の官能基を取得
        left_token, right_token = MON_DEF[key]
        left_ends.append(token2fg[left_token])
        right_ends.append(token2fg[right_token])
    
    # 最初のモノマーから開始
    polymer = mols[0]
    current_right = right_ends[0]
    
    # モノマーを順次結合
    for i in range(1, len(seq)):
        next_mol = mols[i]
        next_left = left_ends[i]
        next_right = right_ends[i]
        
        reacted = False
        
        # 現在のポリマーの右端と次のモノマーの左端に基づいて反応
        if current_right == 'COOH' and next_left == 'PhOH':
            prods = rxn_ester.RunReactants((polymer, next_mol))
            if prods:
                polymer = get_largest_fragment(prods[0][0])
                reacted = True
                
        elif current_right == 'COOH' and next_left == 'NH2':
            prods = rxn_amide.RunReactants((polymer, next_mol))
            if prods:
                polymer = get_largest_fragment(prods[0][0])
                reacted = True
                
        elif current_right == 'PhOH' and next_left == 'COOH':
            prods = rxn_ester.RunReactants((next_mol, polymer))
            if prods:
                polymer = get_largest_fragment(prods[0][0])
                reacted = True
                
        elif current_right == 'NH2' and next_left == 'COOH':
            prods = rxn_amide.RunReactants((next_mol, polymer))
            if prods:
                polymer = get_largest_fragment(prods[0][0])
                reacted = True
        
        if not reacted:
            # 反応しない場合は単純結合
            polymer = get_largest_fragment(Chem.CombineMols(polymer, next_mol))
        
        # 現在の右端を更新
        current_right = next_right
    
    return Chem.MolToSmiles(polymer)

def generate_psmiles(smiles):
    if not smiles:
        return ""
    
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return ""
    
    for rx in [rx_acid, rx_ph, rx_amine]:
        while True:
            prods = rx.RunReactants((mol,))
            if not prods:
                break
            mol = get_largest_fragment(prods[0][0])
    
    return Chem.MolToSmiles(mol)

# --- 5. 実行＆出力 ---
print("シミュレーション開始...")
segment_counter = Counter()
monomer_counter = Counter()

for i in range(N_CHAINS):
    chain = grow_chain()
    monomer_counter.update([key.split('-')[1] for key in chain])
    tokens = tok_chain(chain)
    segments = extract(tokens)
    segment_counter.update(segments)

# モノマー組成チェック
print("\n── モノマー組成チェック ──")
total_monomers = N_CHAINS * CHAIN_LEN
for monomer in monomer_names:
    expected = feed_ratios[monomer]
    observed = monomer_counter[monomer] / total_monomers
    print(f"{monomer}: expected {expected:.4f} | observed {observed:.4f}")

# セグメント処理
print("\nセグメント処理中...")
total_segments = sum(segment_counter.values())
cumulative_prob = 0.0
results = []

for segment, count in segment_counter.most_common():
    prob = count / total_segments
    cumulative_prob += prob
    
    # セグメントからモノマー列を再構築
    tokens = segment.split('-')
    mono_sequence = []
    for i in range(0, len(tokens), 3):
        if i+3 > len(tokens):
            break
        mono_key = '-'.join(tokens[i:i+3])
        if mono_key in MONOMERS:
            mono_sequence.append(mono_key)
    
    # ポリマー構築とpSMILES生成
    try:
        smiles = build_polymer(mono_sequence)
        psmiles_val = generate_psmiles(smiles) if smiles else ""
    except Exception as e:
        print(f"エラー: {e}, セグメント: {segment}")
        smiles, psmiles_val = "", ""
    
    results.append({
        "Pattern": segment,
        "Prob": f"{prob:.3%}",
        "Cum": f"{cumulative_prob:.3%}",
        "Monomers": mono_sequence,
        "SMILES": smiles,
        "p-SMILES": psmiles_val
    })
    
    if cumulative_prob >= TARGET_CUM:
        break

# 結果表示
print(f"\n── 上位 {len(results)} パターン (累積 {TARGET_CUM*100:.0f}%) ──")
df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)
print(df.to_string(index=False))

シミュレーション開始...

── モノマー組成チェック ──
A: expected 0.5000 | observed 0.4991
B: expected 0.1250 | observed 0.1251
C: expected 0.2000 | observed 0.2012
D: expected 0.0500 | observed 0.0504
E: expected 0.1250 | observed 0.1243

セグメント処理中...

── 上位 58 パターン (累積 95%) ──
                                  Pattern    Prob     Cum                                          Monomers                                                                                                            SMILES                                                                                                            p-SMILES
                              a-A-b-a-A-b 25.219% 25.219%                                    [a-A-b, a-A-b]                                                                                 O=C(O)c1ccc(OC(=O)c2ccc(O)cc2)cc1                                                                                 *OC(=O)c1ccc(OC(=O)c2ccc(O*)cc2)cc1
                              a-C-a-b-B-b 10.107% 35.326%          

[14:32:58] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 2 
[14:32:58] product atom-mapping number 3 not found in reactants.
[14:32:58] product atom-mapping number 3 not found in reactants.
[14:32:58] product atom-mapping number 2 not found in reactants.
[14:32:58] mapped atoms in the reactants were not mapped in the products.
  unmapped numbers are: 2 


In [5]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from mordred import Calculator, descriptors
# ---------- 前処理 ----------
# パーセンテージ表記 → 小数に変換
df["Prob_float"] = df["Prob"].str.rstrip('%').astype(float) / 100

# RDKit Mol への変換
mols = []
for smi in df["p-SMILES"]:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, randomSeed=42)
        AllChem.UFFOptimizeMolecule(mol)
        mols.append(mol)
    else:
        mols.append(None)

# 無効なMolは除外
valid_idx = [i for i, mol in enumerate(mols) if mol is not None]
mols = [mols[i] for i in valid_idx]
df_valid = df.iloc[valid_idx].reset_index(drop=True)

# ---------- 記述子の計算 ----------
calc = Calculator(descriptors, ignore_3D=False)
desc_df = calc.pandas(mols).fillna(0)

display(desc_df)
# ---------- 加重平均 ----------
# ② 各列が「数値かつ欠損値がないか」を確認
valid_cols = [
    col for col in desc_df.columns
    if pd.api.types.is_numeric_dtype(desc_df[col]) and not desc_df[col].isnull().any()
]

# ③ 安全な列だけ取り出す
desc_df_valid = desc_df[valid_cols].astype(float)

# ④ 加重平均の計算
desc_df_weighted = (desc_df_valid.T * df_valid["Prob_float"].values).T
weighted_average_valid = desc_df_weighted.sum()

# ⑤ 元の列に対して、使えなかったカラムを空欄（NaN）で埋める
final_result = pd.Series(index=desc_df.columns, dtype="float64")
final_result[valid_cols] = weighted_average_valid
result_df = final_result.to_frame().T
result_df.index = ["WeightedAverage"]
print(df_valid["Prob_float"])
print(result_df.T)

[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (16)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (16)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (16)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (16)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (16)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (16)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (16)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (16)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (23)
[14:55:22] UFFTYPER: Unrecognized atom type: *_ (0)
[14

/home/suehara/miniconda3/envs/polygpu/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/suehara/miniconda3/envs/polygpu/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 58/58 [00:03<00:00, 15.08it/s]


/home/suehara/miniconda3/envs/polygpu/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/suehara/miniconda3/envs/polygpu/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/suehara/miniconda3/envs/polygpu/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/suehara/miniconda3/envs/polygpu/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/suehara/miniconda3/envs/polygpu/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out,

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,15.694249,12.666090,0,0,27.074237,2.319378,4.638755,27.074237,1.289249,3.946074,...,9.704122,54.263214,256.037173,8.828868,1082,30,102.0,116.0,7.416667,4.888889
1,15.694249,12.669921,0,0,27.075759,2.324173,4.648345,27.075759,1.289322,3.946074,...,9.704122,54.263214,256.037173,8.828868,1080,30,102.0,116.0,7.416667,4.888889
2,15.694249,12.669921,0,0,27.075759,2.324173,4.648345,27.075759,1.289322,3.946074,...,9.704122,54.263214,255.053158,8.501772,1080,30,102.0,116.0,7.416667,4.888889
3,15.694249,12.669921,0,0,27.075759,2.324173,4.648345,27.075759,1.289322,3.946074,...,9.704122,54.263214,255.053158,8.501772,1080,30,102.0,116.0,7.416667,4.888889
4,22.834266,16.555251,0,0,38.861202,2.343261,4.686522,38.861202,1.295373,4.309726,...,10.116379,65.232611,376.058303,8.953769,3095,45,150.0,172.0,10.000000,6.833333
5,19.189342,14.381892,0,0,32.760751,2.408974,4.817949,32.760751,1.310430,4.140126,...,10.057710,59.620233,306.052823,8.744366,1710,39,128.0,149.0,8.138889,5.666667
6,22.834266,16.555251,0,0,38.861202,2.343261,4.686522,38.861202,1.295373,4.309726,...,10.116379,65.232611,376.058303,8.953769,3095,45,150.0,172.0,10.000000,6.833333
7,29.974284,20.016978,0,0,50.640683,2.350324,4.700649,50.640683,1.298479,4.575827,...,10.407440,75.641334,496.079432,9.019626,6746,60,198.0,228.0,12.583333,8.777778
8,22.834266,16.555251,0,0,38.861202,2.343261,4.686522,38.861202,1.295373,4.309726,...,10.116379,65.232611,375.074287,8.722658,3095,45,150.0,172.0,10.000000,6.833333
9,22.834266,16.555251,0,0,38.861202,2.343261,4.686522,38.861202,1.295373,4.309726,...,10.116379,65.232611,375.074287,8.722658,3095,45,150.0,172.0,10.000000,6.833333


0     0.25219
1     0.10107
2     0.05100
3     0.05013
4     0.05012
5     0.02542
6     0.02525
7     0.02508
8     0.02501
9     0.02481
10    0.02416
11    0.01266
12    0.01265
13    0.01264
14    0.01261
15    0.01260
16    0.01259
17    0.01249
18    0.01242
19    0.01237
20    0.01232
21    0.01231
22    0.01218
23    0.00643
24    0.00623
25    0.00622
26    0.00621
27    0.00620
28    0.00620
29    0.00619
30    0.00618
31    0.00616
32    0.00615
33    0.00613
34    0.00608
35    0.00603
36    0.00599
37    0.00319
38    0.00318
39    0.00315
40    0.00314
41    0.00313
42    0.00312
43    0.00312
44    0.00310
45    0.00310
46    0.00309
47    0.00309
48    0.00308
49    0.00307
50    0.00302
51    0.00300
52    0.00299
53    0.00296
54    0.00293
55    0.00167
56    0.00156
57    0.00155
Name: Prob_float, dtype: float64
          WeightedAverage
ABC             20.675716
ABCGG           14.966730
nAcid            0.000000
nBase            0.000000
SpAbs_A         35.257386